In [1]:
import heeps
from heeps.util.coord import mas2rms
from heeps.contrast.background import background
import astropy.units as u
from astropy.io import fits
import os.path
from copy import deepcopy

%pylab
%matplotlib inline

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [2]:
conf = dict(
    band = 'L',
    mode = 'CVC',
    dir_current = '/Users/cdelacroix/INSTRUMENTS/METIS/heeps_analysis',
    dir_output = 'qacits',
    file_pupil = 'pupil/ELT_fullM1.fits',
    file_lyot_stop = 'pupil/ls_LM_CVC_circ_285.fits',
    file_phase = 'wavefront/cube_Cbasic_20201130_3600s_300ms_0piston_meters_scao_only_285.fits',
    file_point_err = 'wavefront/point_all_3600s_300ms.fits',
    add_phase = True,
    add_point_err = True,
    nframes = 220,
    cube_duration = 220*0.3,
)
conf = heeps.config.read_config(verbose=False, **conf)
conf = heeps.config.update_config(saveconf=True, verbose=True, **conf)
wf = heeps.pupil.pupil(savefits=True, verbose=True, **conf)
phase_screens, amp_screens, tiptilts, misaligns = heeps.wavefront.load_errors(verbose=True, **conf)

Simulation config: band=L, mode=CVC
‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾
   loading vortex back-propagation params
   vc_charge=2, ngrid=1024, beam_ratio=0.2568
   npupil=285, pscale=5.47 mas, lam=3.8112E-06 m
   hfov=1.1 arcsec (-> ndet=403, 51.74 lam/D)

Load pupil from 'ELT_fullM1.fits'
Load phase screens from 'cube_Cbasic_20201130_3600s_300ms_0piston_meters_scao_only_285.fits'
   nscreens=220 (nframes=220, nstep=1)
Load pointing errors from 'point_all_3600s_300ms.fits'
   nscreens=220


In [3]:
lam = 3.8112E-06
diam = 39.9988 # pupil_img_size
lamD = lam/diam*u.rad.to('mas')
# jusqu’à 0.1 lam/D, step = 1/100
# jusqu’à 1 lam/D, step = 1/10
xy = np.append(np.arange(100)/1000, np.arange(1,11)/10)
print(xy.shape, xy)

(110,) [0.    0.001 0.002 0.003 0.004 0.005 0.006 0.007 0.008 0.009 0.01  0.011
 0.012 0.013 0.014 0.015 0.016 0.017 0.018 0.019 0.02  0.021 0.022 0.023
 0.024 0.025 0.026 0.027 0.028 0.029 0.03  0.031 0.032 0.033 0.034 0.035
 0.036 0.037 0.038 0.039 0.04  0.041 0.042 0.043 0.044 0.045 0.046 0.047
 0.048 0.049 0.05  0.051 0.052 0.053 0.054 0.055 0.056 0.057 0.058 0.059
 0.06  0.061 0.062 0.063 0.064 0.065 0.066 0.067 0.068 0.069 0.07  0.071
 0.072 0.073 0.074 0.075 0.076 0.077 0.078 0.079 0.08  0.081 0.082 0.083
 0.084 0.085 0.086 0.087 0.088 0.089 0.09  0.091 0.092 0.093 0.094 0.095
 0.096 0.097 0.098 0.099 0.1   0.2   0.3   0.4   0.5   0.6   0.7   0.8
 0.9   1.   ]


In [4]:
xy_rms = mas2rms(xy*lamD, diam) # converted to mas, then to rms
tt = [(x,0) for x in xy_rms] + [(0,y) for y in xy_rms]
psfs = []
for i,tiptilt in enumerate(tt):
    psfs.append(heeps.wavefront.propagate_one(wf, tiptilt=tiptilt, tag=i, onaxis=True, verbose=False, **conf))
    if not i % 10:
        print(i, np.sum(psfs[-1]))
fits.writeto('calib.fits', np.float32(psfs), overwrite=True)

0 0.01893186459820351
10 0.019008259299655363
20 0.019237349493310008
30 0.019618852746548555
40 0.020152298991008337
50 0.02083703106564979
60 0.021672206306234582
70 0.022656797585547995
80 0.023789595347452362
90 0.025069209523960034
100 0.026494072175885498
110 0.01893186459820351
120 0.01900828726699721
130 0.019237463359609564
140 0.01961911030882707
150 0.020152757877077234
160 0.02083774876534673
170 0.021673239755701165
180 0.022658203414145647
190 0.02379142954423903
200 0.025071527526139118
210 0.02649692858961276


In [5]:
def create_psfs():
    psf = heeps.wavefront.propagate_one(wf, onaxis=False, savefits=True, verbose=True, **conf)
    psfs = heeps.wavefront.propagate_cube(wf, phase_screens=phase_screens, \
        amp_screens=amp_screens, tiptilts=tiptilts, misaligns=misaligns, onaxis=True, \
        savefits=True, verbose=True, **conf)
    print('mag', end='')
    for conf['mag'] in np.arange(5, 12+1):
        print(', %s'%conf['mag'], end='')
        psf_ON, psf_OFF = background(deepcopy(psfs), deepcopy(psf), **conf)
        fits.writeto(os.path.join(conf['dir_output'], 'onaxis_PSF_L_CVC_mag%s.fits'%conf['mag']), \
            np.float32(psf_ON), overwrite=True)
        fits.writeto(os.path.join(conf['dir_output'], 'offaxis_PSF_L_CVC_mag%s.fits'%conf['mag']), \
            np.float32(psf_OFF), overwrite=True)

In [6]:
conf['dir_output'] = os.path.join(conf['dir_current'], 'qacits/random_drift')
create_psfs()

Create single off-axis PSF
   apply lyot stop from 'ls_LM_CVC_circ_285.fits'
   extract PSF on the detector: ndet=403
Create on-axis PSF cube
   2021-03-01 12:41:52: e2e simulation starts, using 10 cores
   2021-03-01 12:43:20: finished, elapsed 88.038 seconds
mag, 5, 6, 7, 8, 9, 10, 11, 12

In [7]:
conf['dir_output'] = os.path.join(conf['dir_current'], 'qacits/linear_offsets')
tiptilts = tt
create_psfs()

Create single off-axis PSF
   apply lyot stop from 'ls_LM_CVC_circ_285.fits'
   extract PSF on the detector: ndet=403
Create on-axis PSF cube
   2021-03-01 12:43:37: e2e simulation starts, using 10 cores
   2021-03-01 12:45:12: finished, elapsed 95.519 seconds
mag, 5, 6, 7, 8, 9, 10, 11, 12

In [8]:
conf['dir_output'] = os.path.join(conf['dir_current'], 'qacits/scao_only')
tiptilts = np.array([None]*conf['nframes'])
create_psfs()

Create single off-axis PSF
   apply lyot stop from 'ls_LM_CVC_circ_285.fits'
   extract PSF on the detector: ndet=403
Create on-axis PSF cube
   2021-03-01 12:45:33: e2e simulation starts, using 10 cores
   2021-03-01 12:47:06: finished, elapsed 92.739 seconds
mag, 5, 6, 7, 8, 9, 10, 11, 12